# Engenharia de Dados Industrial: SQL e ETL

**Autor:** Davi Duarte Cucco

**Contexto:** Disciplina de Coleta e Tratamento de Dados Inteligentes.

**Objetivo:** Estruturar um banco de dados relacional para substituir planilhas soltas, garantindo integridade e qualidade dos dados de sensores IoT.

**Etapas do Pipeline:**
1.  **Modelagem:** Criação de tabelas Relacionais (Máquinas e Sensores).
2.  **Ingestão:** Inserção de dados simulados de chão de fábrica.
3.  **Analytics:** Consultas SQL complexas com `JOIN` e `GROUP BY` para gerar KPIs.
4.  **Sanitização:** Limpeza automática de dados sujos (erros de leitura e NULLs) usando condicionais SQL.

In [3]:
import sqlite3
import pandas as pd

#Cria o arquivo do banco de dados (se não existir)
conexao = sqlite3.connect('fabrica_inteligente.db')
cursor = conexao.cursor()

In [4]:
#DDL (Data Definition Language): Criando a estrutura das tabelas
#Tabela de Cadastro das Máquinas
cursor.execute('''
CREATE TABLE IF NOT EXISTS Maquinas (
  id_maquina INTEGER PRIMARY KEY,
  modelo TEXT NOT NULL,
  ano_fabricacao INTEGER,
  setor TEXT
)
''')

#Tabela de leitura de sensores
cursor.execute('''
CREATE TABLE IF NOT EXISTS Historico_Sensores(
  id_leitura INTEGER PRIMARY KEY AUTOINCREMENT,
  id_maquina INTEGER,
  data_leitura DATETIME,
  temperatura REAL,
  vibracao REAL,
  FOREIGN KEY (id_maquina) REFERENCES Maquinas(id_maquina)
)
''')

print("Banco de Dados 'fabrica_inteligente.db' criado!")
print("Tabela 'Maquinas' e 'Historico_Sensores' prontas.")

Banco de Dados 'fabrica_inteligente.db' criado!
Tabela 'Maquinas' e 'Historico_Sensores' prontas.


In [5]:
#DML (Data Manipulation Language): Inserindo dados

#Inserindo Máquinas
cursor.execute("INSERT INTO Maquinas VALUES (101, 'Torno CNC',2018,'Usinagem')")
cursor.execute("INSERT INTO Maquinas VALUES (102,'Prensas Hidráulicas',2015,'Estamparia')")
cursor.execute("INSERT INTO Maquinas VALUES (103, 'Braço Robótico',2021,'Montagem')")

#Simulação da leitura do sensores (Imagine que isso vem do IoT)
dados_sensores = [
    (101, '2023-10-01 08:00:00', 75.5, 2.1),
    (101, '2023-10-01 09:00:00', 78.2, 2.3), #Torno esquentando
    (102, '2023-10-01 08:00:00', 60.0, 0.5),
    (103, '2023-10-01 08:00:00', 45.0, 0.1),
    (101, '2023-10-01 10:00:00', 92.0, 5.5) #Torno CRÍTICO
]

#Inserir várias linhas de uma vez
cursor.executemany('''
INSERT INTO Historico_Sensores (id_maquina, data_leitura, temperatura, vibracao)
VALUES (?, ?, ?, ?)
''', dados_sensores)

#Salvar alterações (Commit)
conexao.commit()
print("Dados inseridos com sucesso.")

Dados inseridos com sucesso.


Se o chefe pergunta: "Quais máquinas do setor de Usinagem tiveram temperatura acima de 80 graus?"

No Excel, seria necessário abrir planilhas gigantes e filtrar. No SQL, você faz uma pergunta (Query).

O Pandas tem uma função mágica chamada read_sql que permite rodar SQL e já receber o resultado formatado.

In [6]:
#DQL (Data Query Language): Extraindo Inteligência
#A Query: "Junte a tabela de sensores com a de máquinas, filtre quem esquentou e mostre"
query = '''
SELECT
  m.modelo,
  m.setor,
  s.data_leitura,
  s.temperatura,
  s.vibracao
FROM Historico_Sensores s
JOIN Maquinas m ON s.id_maquina = m.id_maquina
WHERE s.temperatura > 80
ORDER BY s.temperatura DESC
'''
#Executar a query e jo0gar direto num DataFrame
df_alerta = pd.read_sql(query,conexao)

print(" --- RELATÓRIO DE ALERTA DE TEMPERATURA (SQL) --- ")
display(df_alerta)

 --- RELATÓRIO DE ALERTA DE TEMPERATURA (SQL) --- 


,modelo,setor,data_leitura,temperatura,vibracao
0,Torno CNC,Usinagem,2023-10-01 10:00:00,92.0,5.5


JOIN: Cruzou dados de cadastro (Modelo, Setor) com dados operacionais (Temperatura) usando uma chave comum (id_maquina).

O seu chefe não quer ver linha por linha cada vez que o sensor piscou. Ele quer saber: "Qual foi a temperatura média de cada máquina hoje?" "Qual foi o pico máximo de vibração de cada setor?"

No Excel, seria feita uma Tabela Dinâmica. No SQL, usa-se GROUP BY.

In [7]:
#Montando o relatório de KPI's direto do Banco de Dados
#Para cada máquina:
#A média da temperatura,
#O máximo de vibrição,
#Quantas leituras tivemos

query_kpi = '''
SELECT
  m.modelo,
  COUNT(s.id_leitura) as total_leituras,
  AVG(s.temperatura) as temp_media,
  MAX(s.vibracao) as vibracao_maxima
FROM Historico_Sensores s
JOIN Maquinas m ON s.id_maquina = m.id_maquina
GROUP BY m.modelo
ORDER BY vibracao_maxima DESC
'''
df_kpi = pd.read_sql(query_kpi, conexao)

print(" --- RELATÓRIO DE SAÚDE DAS MÁQUINAS (KPIs) --- ")
display(df_kpi)

 --- RELATÓRIO DE SAÚDE DAS MÁQUINAS (KPIs) --- 


,modelo,total_leituras,temp_media,vibracao_maxima
0,Torno CNC,3,81.9,5.5
1,Prensas Hidráulicas,1,60.0,0.5
2,Braço Robótico,1,45.0,0.1


WHERE: Filtra as linhas ANTES de agrupar (ex: "Quero ver apenas dados de 2023").

HAVING: Filtra o resultado DEPOIS de agrupar (ex: "Quero ver apenas as máquinas cuja Média foi maior que 70°C").

In [8]:
#Filtrando Grupos (HAVING)
query_criticas = '''
SELECT
  m.modelo,
  AVG(s.temperatura) as temp_media
FROM Historico_Sensores s
JOIN Maquinas m ON s.id_maquina = m.id_maquina
GROUP BY m.modelo
HAVING AVG(s.temperatura) > 50
'''

df_criticas = pd.read_sql(query_criticas, conexao)

print(" --- MÁQUINAS EM ESTADO DE ALERTA (HAVING > 50°C) --- ")
display(df_criticas)

 --- MÁQUINAS EM ESTADO DE ALERTA (HAVING > 50°C) --- 


,modelo,temp_media
0,Prensas Hidráulicas,60.0
1,Torno CNC,81.9


Kit básico de sobrevivência em SQL:

CREATE/INSERT: Criar e salvar dados.

SELECT/JOIN: Cruzar tabelas.

GROUP BY/HAVING: Gerar relatórios e métricas.

Na fábrica, o sensor falha, a internet cai, ou alguém tropeça no cabo. O resultado no banco de dados é desastroso:

NULL: O campo fica vazio (buraco no dado).

Outliers/Erros: Um forno a -500ºC (fisicamente impossível).

É necessário efetuar uma limpeza nos dados para utilizá-los nos KPIs.

In [9]:
#Inserindo dados sujos

dados_ruins=[
    (102, '2023-10-01 11:00:00', 62.5, None), #1 - Sensor de vibração falhou mandando NULL (None)
    (103, '2023-10-01 11:00:00', -999.0, 0.2) #2 - Sensor de temperatura deu bug e mandou -999 (impossível)
]

cursor.executemany('''
INSERT INTO Historico_Sensores (id_maquina, data_leitura, temperatura, vibracao)
VALUES (?, ?, ?, ?)
''', dados_ruins)
conexao.commit()
print("Dados sujos inseridos.")

Dados sujos inseridos.


É necessário escrever uma query que não altera o banco original (regra de ouro da Engenharia de Dados: nunca apague o dado bruto), mas entrega uma "visão" limpa para o analista.

Dois comandos vitais do SQL:

COALESCE(coluna, valor_padrao): Se for NULL, preenche com zero (ou média).

CASE WHEN ... THEN ... END: É o "IF/ELSE" do SQL para corrigir lógica.

In [10]:
#Tratamento de Dados Inteligente com SQL

query_limpeza = '''
SELECT
  id_leitura,
  id_maquina,

  -- Tratamento 1: O problema do -999
  -- Se temp < 0, considera erro e mostra NULL (ou média)
  CASE
    WHEN temperatura < 0 THEN 'ERRO_LEITURA'
    ELSE temperatura
  END as temp_tratada,

  -- Tratamento 2: O problema do NULL (Vibração vazia)
  -- Se for NULL, assumimos 0.0 (máquina parada/sensor off)
  COALESCE(vibracao, 0.0) as vibracao_tratada,

  -- Mantendo os dados originais para auditoria
  temperatura as temp_original,
  vibracao as vibracao_original

FROM Historico_Sensores
WHERE id_leitura > 5 -- Só para ver os novos dados
'''

df_limpo = pd.read_sql(query_limpeza, conexao)

print(" --- DADOS TRATADOS (PRONTOS PARA MACHINE LEARNING) --- ")
display(df_limpo)

 --- DADOS TRATADOS (PRONTOS PARA MACHINE LEARNING) --- 


,id_leitura,id_maquina,temp_tratada,vibracao_tratada,temp_original,vibracao_original
0,6,102,62.5,0.0,62.5,NaN
1,7,103,ERRO_LEITURA,0.2,-999.0,0.2


Na coluna temp_tratada: Onde era -999.0, agora está escrito ERRO_LEITURA (ou poderia colocar 0 ou a média da máquina).

Na coluna vibracao_tratada: Onde era None (NaN), agora é 0.0.



In [11]:
#Encerrando os trabalhos
#É preciso fechar a conexão com o banco para não corromper o arquivo
conexao.close()
print("Conexão com o Banco de Dados fechada.")

Conexão com o Banco de Dados fechada.


Isso tudo foi a execução de um Pipeline de ETL (Extract, Transform, Load) completo dentro do notebook:

Criou o Banco (Modelagem).

Inseriu Dados (Ingestão).

Limpou e Transformou (Tratamento).

# Conclusão e Impacto na Engenharia

## Para que serve tudo isso?
Tudo isso criado não é apenas um "lugar para guardar dados". É um **Pipeline de Engenharia de Dados**.
Na prática industrial, isso serve para:
1.  **Eliminar o "Inferno das Planilhas":** Substituir arquivos Excel manuais e descentralizados (que corrompem e têm versões conflitantes) por uma **Fonte Única da Verdade (Single Source of Truth)**.
2.  **Garantir a Qualidade na Fonte:** Ao usarmos SQL para limpar dados sujos (erros de leitura e nulos) *antes* de eles chegarem aos analistas, garantimos que qualquer relatório ou IA gerado a partir daqui seja confiável.
3.  **Auditabilidade:** Diferente de uma planilha onde alguém pode apagar um número sem querer, o Banco de Dados mantém o registro histórico seguro e estruturado.

## O que é possível concluir?
Através das queries executadas, concluímos que:
* **O SQL é uma ferramenta analítica, não só de armazenamento:** Conseguimos gerar KPIs complexos (como média de temperatura por setor) e filtrar máquinas críticas sem precisar abrir o Python ou Excel.
* **Dados brutos mentem:** A simulação mostrou que sensores falham (temperaturas negativas, nulos). Sem a etapa de **Sanitização (Data Cleaning)** que implementamos, qualquer decisão tomada estaria errada.
* **Performance:** Processar milhões de linhas no SQL é infinitamente mais rápido e leve do que carregar tudo na memória RAM do computador.

## Aplicações Práticas deste Projeto
Esta estrutura é a base para sistemas reais como:
* **Dashboards de Gestão à Vista:** Conectar este banco ao **Power BI ou Grafana** para mostrar a vibração das máquinas em telões na fábrica em tempo real.
* **Cálculo de OEE Automático:** Usar os dados de tempo e produção para calcular a eficiência global das máquinas automaticamente.
* **Alimentação de Modelos de IA:** Este banco é quem fornece os dados limpos para os modelos de *Manutenção Preditiva* (que calculam o RUL - Tempo de Vida Útil).

## Outros Contextos de Uso
A lógica de SQL e ETL aplicada aqui é universal:
* **Logística:** Rastreamento de frota (em vez de sensores de máquina, temos GPS de caminhões).
* **Varejo:** Análise de cestas de compras (quais produtos saem juntos) e gestão de estoque.
* **Finanças:** Detecção de fraudes (limpeza de transações duplicadas ou suspeitas).
* **Saúde:** Prontuário eletrônico unificado, cruzando dados de exames com histórico do paciente.